# Configuration
# - Python 3.13

In [ ]:
!pip3 install pandas seaborn numpy matplotlib scikit-learn xgboost --quiet

In [ ]:
!pip3 freeze > requirements.txt

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, KFold
import random as rn
RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
# ==========================================================
# 평가함수
# ==========================================================

def smape(gt, preds):
    gt= np.array(gt)
    preds = np.array(preds)
    v = 2 * abs(preds - gt) / (abs(preds) + abs(gt))
    score = np.mean(v) * 100
    return score

def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def inverse_value_weighted_mse(label, pred):
    """
    실제값(label)이 작을수록 더 큰 가중치를 부여하는 목적 함수
    """
    # 안전장치 epsilon 추가
    epsilon = 1e-6
    residual = (label - pred).astype("float")

    # 실제값의 크기에 반비례하는 가중치 계산
    # label이 0에 가까울수록 weight는 매우 커짐
    weight = 1.0 / (label + epsilon)

    # 가중치를 적용한 gradient와 hessian 계산
    grad = -2.0 * weight * residual
    hess = 2.0 * weight

    return grad, hess

def custom_huber(alpha=1.2, delta=1.0):
    '''
    과소예측에 더 큰 패널티를 부여하는 huber 목적 함수
    '''
    def asymmetric_huber(label, pred):
        alpha = alpha
        delta = delta

        residual = label - pred
        abs_residual = np.abs(residual)
        huber_grad = np.where(abs_residual <= delta, residual, delta * np.sign(residual))

        grad = np.where(residual > 0, alpha*huber_grad, huber_grad)
        hess = np.where(residual > 0, alpha, 1.0)

        return grad, hess
    return asymmetric_huber

def custom_smape(preds, labels):
    smape = np.mean(2 * abs(preds - labels) / (abs(preds) + abs(labels))) * 100
    return smape

# data load

In [ ]:
train = pd.read_csv('./data/raw/train.csv')
test = pd.read_csv('./data/raw/test.csv')
building_info = pd.read_csv('./data/raw/building_info.csv')

# data reindexing

In [ ]:
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train.drop('num_date_time', axis = 1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test.drop('num_date_time', axis = 1, inplace=True)

building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '학교': 'University',
    '백화점': 'Department Store',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    'IDC(전화국)': 'IDC',
    '호텔': 'Hotel'
}

building_info['building_type'] = building_info['building_type'].replace(translation_dict)

building_info['solar_power_utility_binary'] = np.where(building_info.solar_power_capacity !='-',1,0)
building_info['ess_utility_binary'] = np.where(building_info.ess_capacity !='-',1,0)

building_info['solar_power_capacity'] = building_info['solar_power_capacity'].replace('-', '0').astype(float)
building_info['ess_capacity'] = building_info['ess_capacity'].replace('-', '0').astype(float)

train = pd.merge(train, building_info, on='building_number', how='left')
test = pd.merge(test, building_info, on='building_number', how='left')

# Outlier Marking

In [ ]:
# outlier_detect 컬럼 초기화 (0: 정상, 1: 이상치)
train['outlier_detect'] = 0
test['outlier_detect'] = 0

# power_consumption이 0인 경우를 이상치로 마킹 (앞뒤 1포인트 포함)
zero_outlier_indices = train.index[train['power_consumption'] == 0].tolist()

# 앞뒤 1포인트도 포함하여 확장된 이상치 인덱스 생성
expanded_outlier_indices = set()
for idx in zero_outlier_indices:
    # 현재 인덱스와 앞뒤 1포인트 추가
    for offset in [-1, 0, 1]:
        new_idx = idx + offset
        if new_idx >= 0 and new_idx < len(train):  # 인덱스 범위 체크
            expanded_outlier_indices.add(new_idx)

# 확장된 인덱스들을 이상치로 마킹
train.loc[list(expanded_outlier_indices), 'outlier_detect'] = 1

# 이상치 개수 확인
outlier_count = (train['outlier_detect'] == 1).sum()
total_count = len(train)
print(f"총 데이터 개수: {total_count}")
print(f"power_consumption이 0인 원본 이상치 개수: {len(zero_outlier_indices)}")
print(f"앞뒤 1포인트 포함 확장된 이상치 개수: {outlier_count}")
print(f"확장된 이상치 비율: {outlier_count/total_count*100:.2f}%")

In [ ]:
# IQR 방식을 이용한 이상치 탐지 (앞뒤 1포인트 포함)
def detect_outliers_iqr_expanded(df, column, building_col='building_number'):
    """
    IQR 방식으로 이상치를 탐지하는 함수
    건물별로 따로 계산하여 더 정확한 이상치 탐지
    이상치 발견 시 앞뒤 1포인트도 함께 이상치로 마킹
    """
    outlier_indices = []

    for building in df[building_col].unique():
        building_mask = df[building_col] == building
        building_data = df[building_mask][column]
        building_indices = df[building_mask].index.tolist()
        building_indices_set = set(building_indices)  # 빠른 검색을 위해 set 사용

        # 0이 아닌 값들만으로 IQR 계산 (0은 이미 이상치로 마킹됨)
        non_zero_data = building_data[building_data > 0]

        if len(non_zero_data) > 0:
            Q1 = non_zero_data.quantile(0.25)
            Q3 = non_zero_data.quantile(0.75)
            IQR = Q3 - Q1

            # IQR 기반 경계값 계산
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # 해당 건물의 이상치 인덱스 찾기 (0값은 제외, 이미 마킹됨)
            building_outliers_mask = ((building_data < lower_bound) | (building_data > upper_bound)) & (building_data > 0)
            original_outlier_indices = building_data[building_outliers_mask].index.tolist()

            # 앞뒤 1포인트 확장
            expanded_outlier_indices = set()
            for idx in original_outlier_indices:
                # 현재 인덱스와 앞뒤 1포인트 추가
                for offset in [-1, 0, 1]:
                    new_idx = idx + offset
                    # 인덱스 범위 체크 및 같은 건물 내의 인덱스인지 확인
                    if 0 <= new_idx < len(df) and new_idx in building_indices_set:
                        expanded_outlier_indices.add(new_idx)

            outlier_indices.extend(list(expanded_outlier_indices))

            print(f"건물 {building}: Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}")
            print(f"  Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")
            print(f"  원본 IQR 이상치 개수: {len(original_outlier_indices)}")
            print(f"  확장된 IQR 이상치 개수: {len(expanded_outlier_indices)}")

    return outlier_indices

# power_consumption에 대해 IQR 이상치 탐지 (확장 버전)
print("=== IQR 방식 이상치 탐지 (앞뒤 1포인트 포함) ===")
iqr_outlier_indices = detect_outliers_iqr_expanded(train, 'power_consumption')

# IQR 이상치를 outlier_detect에 마킹 (기존 0값 이상치와 합쳐짐)
train.loc[iqr_outlier_indices, 'outlier_detect'] = 1

# 최종 이상치 개수 확인
total_outliers = (train['outlier_detect'] == 1).sum()
zero_outliers = len(zero_outlier_indices)  # 원본 0값 이상치 개수
iqr_outliers = len(iqr_outlier_indices)

print(f"\n=== 최종 이상치 현황 ===")
print(f"power_consumption=0 원본 이상치: {zero_outliers}개")
print(f"IQR 방식 확장 이상치: {iqr_outliers}개")
print(f"총 이상치 개수: {total_outliers}개")
print(f"전체 데이터 대비 이상치 비율: {total_outliers/len(train)*100:.2f}%")

In [ ]:
# ==========================================================
# 이상치 제거 원상복구
# ==========================================================

# 이상치 제거 원상복구 설정
exclude_outlier_removal_buildings = [1, # 호텔
                                     13, 15, 23, 94, # 연구소
                                     8, 22, 46, 55, 87, # 학교
                                     71, 25, 91, 93, #아파트
                                     2, 51, 99, # 상용
                                     30, 37, 43, 52, 64, # IDC(전화국)
                                     96, # 건물기타
                                     ]  # 이상치 제거를 하지 않을 건물 번호 리스트

# 지정된 건물의 outlier_detect를 0으로 설정 (원상복구)
for building_id in exclude_outlier_removal_buildings:
    if building_id in train['building_number'].unique():
        mask = train['building_number'] == building_id
        train.loc[mask, 'outlier_detect'] = 0
        print(f"Building {building_id}: {mask.sum()} rows restored (outlier_detect set to 0)")
    else:
        print(f"Warning: Building {building_id} not found in dataset")

# 현재 이상치 상태 확인
print("\n=== 현재 이상치 마킹 상태 ===")
outlier_count_by_building = train.groupby('building_number')['outlier_detect'].sum().sort_index()
total_outliers = train['outlier_detect'].sum()
total_rows = len(train)

print(f"전체 데이터: {total_rows:,} rows")
print(f"전체 이상치: {total_outliers:,} rows ({total_outliers/total_rows*100:.2f}%)")
print(f"\n건물별 이상치 개수:")
for building_id, count in outlier_count_by_building.items():
    building_total = len(train[train['building_number'] == building_id])
    print(f"  Building {building_id:2d}: {count:4d} outliers ({count/building_total*100:5.2f}%)")


In [ ]:
# ==========================================================
# 특정 기간 이상치 마킹
# ==========================================================

def mark_outlier_periods(df):
    """
    reference/xgb_type.py에서 확인한 특정 기간들을 이상치로 마킹
    기존에는 데이터를 drop했지만, 여기서는 outlier_detect=1로 마킹

    기간 형식:
    - 날짜만: ['2024-07-07', '2024-07-08']  # 전체 날짜
    - 시간까지: ['2024-07-07 14:00', '2024-07-08 09:00']  # 특정 시간대
    """

    print("=== 특정 기간 이상치 마킹 시작 ===")

    # date_time 컬럼이 datetime 타입인지 확인하고 변환
    if not pd.api.types.is_datetime64_any_dtype(df['date_time']):
        print("date_time을 datetime 타입으로 변환 중...")
        df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d %H')

    initial_outliers = (df['outlier_detect'] == 1).sum()

    # 1. 건물기타 (Other Buildings) 등.
    other_buildings_config = [
        {'building_id': 7, 'periods': [
            ['2024-07-07 10:00', '2024-07-08 11:00'],
            ['2024-07-12 14:00', '2024-08-06 03:00']
        ]},
        {'building_id': 26, 'periods': [
            ['2024-06-17 14:00', '2024-06-18 11:00']
        ]},
        {'building_id': 69, 'periods': [
            ['2024-06-02', '2024-06-02']
        ]},
        {'building_id': 82, 'periods': [
            ['2024-07-17 14:00', '2024-07-17 14:00']
        ]}
    ]

    # 2. IDC(전화국) 등
    idc_config = [
        {'building_id': 30, 'periods': [
            ['2024-07-13 20:00', '2024-07-13 20:00'],
            ['2024-07-25 00:00', '2024-07-25 00:00']
        ]},
        {'building_id': 36, 'periods': [
            ['2024-07-21 00:00', '2024-07-21 23:00']
        ]},
        {'building_id': 43, 'periods': [
            ['2024-06-10 17:00', '2024-06-10 18:00'],
            ['2024-08-12 16:00', '2024-08-12 17:00']
        ]},
        {'building_id': 52, 'periods': [
            ['2024-08-10 00:00', '2024-08-10 02:00']
        ]},
        {'building_id': 57, 'periods': [
            ['2024-06-01', '2024-06-07']
        ]},
        {'building_id': 67, 'periods': [
            ['2024-07-26', '2024-07-27'],
            ['2024-08-01 15:00', '2024-08-01 17:00']
        ]}
    ]

    # 3. 병원 (Hospital) 등.
    hospital_config = [
        {'building_id': 17, 'periods': [
            ['2024-06-25 20:00', '2024-06-26 08:00']
        ]},
        {'building_id': 44, 'periods': [
            ['2024-06-06 12:00', '2024-06-06 14:00']
        ]}
    ]

    # 4. 상용 (Commercial) 등
    commercial_config = [
        {'building_id': 20, 'periods': [
            ['2024-06-01 10:00', '2024-06-01 11:00'],
            ['2024-06-09 10:00', '2024-06-09 10:00']
        ]},
        {'building_id': 41, 'periods': [
            ['2024-07-17 09:00', '2024-07-17 15:00']
        ]},
        {'building_id': 51, 'periods': [
            ['2024-06-30', '2024-06-30']
        ]},
    ]

    # 5. 아파트 (Apartment) 등
    apartment_config = [
        {'building_id': 25, 'periods': [
            ['2024-07-04 12:00', '2024-07-04 14:00']
        ]},
        {'building_id': 65, 'periods': [
            ['2024-06-01', '2024-06-09']
        ]},
        {'building_id': 70, 'periods': [
            ['2024-06-04 09:00', '2024-06-05 08:00']
        ]}
    ]

    # 6. 연구소 (Research Institute) 등.
    research_config = [
        {'building_id': 49, 'periods': [
            ['2024-06-15 09:00', '2024-06-15 11:00'],
            ['2024-07-06', '2024-07-07'],
            ['2024-08-17', '2024-08-18'],
            ['2024-08-22', '2024-08-22'],
        ]},
        {'building_id': 53, 'periods': [
            ['2024-06-14 16:00', '2024-06-17 10:00'],
            ['2024-08-18 15:00', '2024-08-19 09:00']
        ]},
        {'building_id': 83, 'periods': [
            ['2024-07-17 14:00', '2024-07-17 14:00'],
        ]},
        {'building_id': 94, 'periods': [
            ['2024-07-26 18:00', '2024-08-05 05:00']
        ]}
    ]

    # 7. 학교 (University) 등.
    university_config = [
        {'building_id': 8, 'periods': [
            ['2024-07-21 08:00', '2024-07-21 11:00'],
            ['2024-08-24 09:00', '2024-08-24 23:00']
        ]},
        {'building_id': 12, 'periods': [
            ['2024-07-21 08:00', '2024-07-21 11:00'],
            ['2024-08-24 08:00', '2024-08-24 10:00']
        ]},
        {'building_id': 55, 'periods': [
            ['2024-07-17 14:00', '2024-07-17 14:00'],
        ]},
        {'building_id': 87, 'periods': [
            ['2024-06-01', '2024-06-30']
        ]}
    ]

    # 8. 호텔 (Hotel) 등
    hotel_config = [
        {'building_id': 89, 'periods': [
            ['2024-07-12 00:00', '2024-07-12 23:00']
        ]}
    ]

    # 9. 공공 (Public) 등
    public_config = [
        {'building_id': 38, 'periods': [
            ['2024-07-17 14:00', '2024-07-17 15:00']
        ]},
        {'building_id': 72, 'periods': [
            ['2024-06-11 00:00', '2024-06-11 02:00']
        ]},
        {'building_id': 92, 'periods': [
            ['2024-07-17 14:00', '2024-07-18 04:00']
        ]}
    ]

    # 10. 백화점 (Department Store) 등
    department_store_config = [
        {'building_id': 19, 'periods': [
            ['2024-07-31 13:00', '2024-07-31 16:00']
        ]},
        {'building_id': 32, 'periods': [
            ['2024-07-08 09:00', '2024-07-08 10:00']
        ]},
        {'building_id': 40, 'periods': [
            ['2024-07-14 00:00', '2024-07-14 01:00']
        ]},
        {'building_id': 45, 'periods': [
            ['2024-07-04 00:00', '2024-07-04 03:00']
        ]},
        {'building_id': 73, 'periods': [
            ['2024-07-08 22:00', '2024-07-08 22:00']
        ]},
        {'building_id': 79, 'periods': [
            ['2024-08-19 03:00', '2024-08-19 05:00']
        ]},
        {'building_id': 95, 'periods': [
            ['2024-08-05 10:00', '2024-08-05 11:00']
        ]},
        # ===================== 월요일 제거 ==========================

        {'building_id': 19, 'periods': [
            ['2024-06-10 00:00', '2024-06-10 23:00'],
            ['2024-07-08 00:00', '2024-07-08 23:00'],
            ['2024-08-19 00:00', '2024-08-19 23:00'],
        ]},

        {'building_id': 45, 'periods': [
            ['2024-06-10 00:00', '2024-06-10 23:00'],
            ['2024-07-08 00:00', '2024-07-08 23:00'],
            ['2024-08-19 00:00', '2024-08-19 23:00'],
        ]},

        {'building_id': 54, 'periods': [
            ['2024-06-17 00:00', '2024-06-17 23:00'],
            ['2024-07-01 00:00', '2024-07-01 23:00'],
            ['2024-08-19 00:00', '2024-08-19 23:00'],
        ]},

        {'building_id': 74, 'periods': [
            ['2024-06-17 00:00', '2024-06-17 23:00'],
            ['2024-07-01 00:00', '2024-07-01 23:00'],
        ]},

        {'building_id': 79, 'periods': [
            ['2024-06-17 00:00', '2024-06-17 23:00'],
            ['2024-07-01 00:00', '2024-07-01 23:00'],
            ['2024-08-19 00:00', '2024-08-19 23:00'],
        ]},
        {'building_id': 95, 'periods': [
            ['2024-07-08 00:00', '2024-07-08 23:00'],
            ['2024-08-05 00:00', '2024-08-05 23:00'],
        ]},


    ]

    # 모든 설정을 통합 (10개 건물 타입 전체)
    all_configs = [
        ('건물기타 (Other Buildings)', other_buildings_config),
        ('IDC(전화국)', idc_config),
        ('병원 (Hospital)', hospital_config),
        ('상용 (Commercial)', commercial_config),
        ('아파트 (Apartment)', apartment_config),
        ('연구소 (Research Institute)', research_config),
        ('학교 (University)', university_config),
        ('호텔 (Hotel)', hotel_config),
        ('공공 (Public)', public_config),
        ('백화점 (Department Store)', department_store_config)
    ]

    total_marked = 0

    for building_type_name, configs in all_configs:
        print(f"\\n--- {building_type_name} 처리 ---")
        type_marked = 0

        if not configs:  # 빈 리스트인 경우
            print(f"  {building_type_name}: 설정된 이상치 기간 없음")
            continue

        for config in configs:
            building_id = config['building_id']
            periods = config['periods']

            building_marked = 0
            for period in periods:
                # 시간이 포함되어 있는지 확인
                if len(period[0].split()) > 1:  # 시간이 포함된 경우
                    start_datetime = pd.to_datetime(period[0])
                    end_datetime = pd.to_datetime(period[1])

                    # 정확한 시간까지 비교
                    condition = (df['building_number'] == building_id) & \
                               (df['date_time'] >= start_datetime) & \
                               (df['date_time'] <= end_datetime)

                    print(f"  건물 {building_id}: {period[0]} ~ {period[1]} (시간 포함)", end=" | ")
                else:  # 날짜만 있는 경우 (기존 방식)
                    start_date = pd.to_datetime(period[0]).date()
                    end_date = pd.to_datetime(period[1]).date()

                    # 해당 건물과 기간에 맞는 조건
                    condition = (df['building_number'] == building_id) & \
                               (df['date_time'].dt.date >= start_date) & \
                               (df['date_time'].dt.date <= end_date)

                    print(f"  건물 {building_id}: {period[0]} ~ {period[1]} (전체 날짜)", end=" | ")

                marked_count = condition.sum()
                df.loc[condition, 'outlier_detect'] = 1
                building_marked += marked_count
                print(f"{marked_count}개 마킹")

            type_marked += building_marked

        total_marked += type_marked
        print(f"  {building_type_name} 총 마킹: {type_marked}개")

    final_outliers = (df['outlier_detect'] == 1).sum()
    newly_marked = final_outliers - initial_outliers

    print(f"\\n=== 특정 기간 이상치 마킹 완료 ===")
    print(f"기존 이상치: {initial_outliers}개")
    print(f"새로 마킹된 이상치: {newly_marked}개")
    print(f"최종 이상치: {final_outliers}개")
    print(f"전체 데이터 대비 이상치 비율: {final_outliers/len(df)*100:.2f}%")

    return df

# date_time 컬럼을 먼저 datetime 타입으로 변환
print("=== 날짜 데이터 타입 확인 및 변환 ===")
if not pd.api.types.is_datetime64_any_dtype(train['date_time']):
    print("train 데이터의 date_time을 datetime 타입으로 변환 중...")
    train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')

if not pd.api.types.is_datetime64_any_dtype(test['date_time']):
    print("test 데이터의 date_time을 datetime 타입으로 변환 중...")
    test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

# train 데이터에 특정 기간 이상치 마킹 적용
train = mark_outlier_periods(train)

# test 데이터에도 outlier_detect 컬럼이 있는지 확인하고 같은 기간 마킹
if 'outlier_detect' in test.columns:
    print("\\n=== Test 데이터에도 같은 기간 이상치 마킹 ===")
    test = mark_outlier_periods(test)
else:
    print("\\nTest 데이터에는 outlier_detect 컬럼이 없어 건너뜁니다.")

# feature engineering

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')

# date time feature 생성
train['hour'] = train['date_time'].dt.hour
train['day'] = train['date_time'].dt.day
train['month'] = train['date_time'].dt.month
train['day_of_week'] = train['date_time'].dt.dayofweek #요일


test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

# date time feature 생성
test['hour'] = test['date_time'].dt.hour
test['day'] = test['date_time'].dt.day
test['month'] = test['date_time'].dt.month
test['day_of_week'] = test['date_time'].dt.dayofweek #요일

In [ ]:
def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
    result_dict = {}

    grouped_temp = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)

    for (building, month, day), value in grouped_temp.items():
        result_dict.setdefault(building, {}).setdefault(month, {})[day] = value

    dataframe[output_column] = [
        result_dict.get(row['building_number'], {}).get(row['month'], {}).get(row['day'], None)
        for _, row in dataframe.iterrows()
    ]


train['day_max_temperature'] = 0.0
train['day_mean_temperature'] = 0.0

calculate_day_values(train, 'temperature', 'day_max_temperature', 'max')
calculate_day_values(train, 'temperature', 'day_mean_temperature', 'mean')
calculate_day_values(train, 'temperature', 'day_min_temperature', 'min')

train['day_temperature_range'] = train['day_max_temperature'] - train['day_min_temperature']

calculate_day_values(test, 'temperature', 'day_max_temperature', 'max')
calculate_day_values(test, 'temperature', 'day_mean_temperature', 'mean')
calculate_day_values(test, 'temperature', 'day_min_temperature', 'min')

test['day_temperature_range'] = test['day_max_temperature'] - test['day_min_temperature']

In [ ]:
# ==========================================================
# 기본 시간 피처
# ==========================================================

# 주기 24시간
train['sin_hour'] = np.sin(2 * np.pi * train['hour']/24.0)
train['cos_hour'] = np.cos(2 * np.pi * train['hour']/24.0)
test['sin_hour'] = np.sin(2 * np.pi * test['hour']/24.0)
test['cos_hour'] = np.cos(2 * np.pi * test['hour']/24.0)

# 주기 7일 (Pandas: 월요일=0 ~ 일요일=6)
train['sin_dayofweek'] = np.sin(2 * np.pi * train['day_of_week']/7.0)
train['cos_dayofweek'] = np.cos(2 * np.pi * train['day_of_week']/7.0)
test['sin_dayofweek'] = np.sin(2 * np.pi * test['day_of_week']/7.0)
test['cos_dayofweek'] = np.cos(2 * np.pi * test['day_of_week']/7.0)

# 데이터 시작점으로부터 얼마나 시간이 흘렀는지
train['time_index'] = (train['date_time'] - train['date_time'].min()).dt.total_seconds() // 3600
test['time_index'] = (test['date_time'] - train['date_time'].min()).dt.total_seconds() // 3600

# 월별
days_in_month_map = {
    6: 30,
    7: 31,
    8: 31
}

train['period'] = train['month'].map(days_in_month_map)
test['period'] = test['month'].map(days_in_month_map)

train['sin_day'] = np.sin(2 * np.pi * train['day'] / train['period'])
train['cos_day'] = np.cos(2 * np.pi * train['day'] / train['period'])

test['sin_day'] = np.sin(2 * np.pi * test['day'] / test['period'])
test['cos_day'] = np.cos(2 * np.pi * test['day'] / test['period'])

train = train.drop('period', axis=1)
test = test.drop('period', axis=1)

In [ ]:
# ==========================================================
# 추가 시간 피처
# ==========================================================

# 주말
for df in [train, test]:
    df["is_weekend"] = 0

    weekend_condition = (
        (df['date_time'].dt.dayofweek == 5) |\
        (df['date_time'].dt.dayofweek == 6)
    )
    df.loc[weekend_condition, 'is_weekend'] = 1

# 공휴일
train['is_holiday'] = 0
test['is_holiday'] = 0

holiday_condition_train = (train['date_time'].dt.month == 6) & (train['date_time'].dt.day == 6) | \
                          (train['date_time'].dt.month == 8) & (train['date_time'].dt.day == 15)

holiday_condition_test = (test['date_time'].dt.month == 6) & (test['date_time'].dt.day == 6) | \
                         (test['date_time'].dt.month == 8) & (test['date_time'].dt.day == 15)

train.loc[holiday_condition_train, 'is_holiday'] = 1
test.loc[holiday_condition_test, 'is_holiday'] = 1

# 백화점 2,4째주 일요일 휴무
train['is_bimonthly_sunday_off'] = 0
test['is_bimonthly_sunday_off'] = 0

is_sunday = train['date_time'].dt.dayofweek == 6
is_2nd_or_4th_week = ((train['date_time'].dt.day - 1) // 7 == 1) | \
                     ((train['date_time'].dt.day - 1) // 7 == 3)

train.loc[is_sunday & is_2nd_or_4th_week, 'is_bimonthly_sunday_off'] = 1

is_sunday_test = test['date_time'].dt.dayofweek == 6
is_2nd_or_4th_week_test = ((test['date_time'].dt.day - 1) // 7 == 1) | \
                          ((test['date_time'].dt.day - 1) // 7 == 3)

test.loc[is_sunday_test & is_2nd_or_4th_week_test, 'is_bimonthly_sunday_off'] = 1

# 백화점 2,4째주 월요일 휴무
train['is_bimonthly_monday_off'] = 0
test['is_bimonthly_monday_off'] = 0

is_monday = train['date_time'].dt.dayofweek == 0
is_2nd_or_4th_week = ((train['date_time'].dt.day - 1) // 7 == 1) | \
                     ((train['date_time'].dt.day - 1) // 7 == 3)

train.loc[is_monday & is_2nd_or_4th_week, 'is_bimonthly_monday_off'] = 1

is_monday_test = test['date_time'].dt.dayofweek == 0
is_2nd_or_4th_week_test = ((test['date_time'].dt.day - 1) // 7 == 1) | \
                          ((test['date_time'].dt.day - 1) // 7 == 3)

test.loc[is_monday_test & is_2nd_or_4th_week_test, 'is_bimonthly_monday_off'] = 1

# 백화점 2째주 월요일 휴무
train['is_monthly_monday_off'] = 0
test['is_monthly_monday_off'] = 0

is_monday = train['date_time'].dt.dayofweek == 0
is_2th_week = ((train['date_time'].dt.day - 1) // 7 == 1)

train.loc[is_monday & is_2th_week, 'is_monthly_monday_off'] = 1

is_monday = test['date_time'].dt.dayofweek == 0
is_2th_week = ((test['date_time'].dt.day - 1) // 7 == 1)

test.loc[is_monday & is_2th_week, 'is_monthly_monday_off'] = 1

# 호텔 성수기
peak_season_config = {
    9: [
        {'start': '2024-07-13', 'end': '2024-08-31'}
    ],
    10: [
        {'start': '2024-07-04', 'end': '2024-08-22'}
    ],
    28: [
        {'start': '2024-07-18', 'end': '2024-08-31'}
    ],
    77: [
        {'start': '2024-07-18', 'end': '2024-08-31'}
    ],
    89: [
        {'start': '2024-07-17', 'end': '2024-08-31'}
    ],
    98: [
        {'start': '2024-07-15', 'end': '2024-08-31'}
    ],
    100: [
        {'start': '2024-07-15', 'end': '2024-08-31'}
    ],
}

for df in [train, test]:
    df['is_peak_season'] = 0

    for building_id, seasons in peak_season_config.items():
        for season in seasons:
            start_date = pd.to_datetime(season['start'])
            end_date = pd.to_datetime(season['end'])

            condition = (
                (df['building_number'] == building_id) &
                (df['date_time'] >= start_date) &
                (df['date_time'] <= end_date)
            )
            df.loc[condition, 'is_peak_season'] = 1

# 연구소 황금연휴
holiday_research = {
    23: [
        {"start": '2024-06-06', 'end': '2024-06-07'},
        {"start": '2024-08-15', 'end': '2024-08-16'}
    ],
    94: [
        {"start": '2024-06-06', 'end': '2024-06-07'},
        {"start": '2024-08-15', 'end': '2024-08-16'}
    ]
}

for df in [train, test]:
    df['is_holiday_research'] = 0

    for building_id, seasons in holiday_research.items():
        for season in seasons:
            start_date = pd.to_datetime(season['start'])
            end_date = pd.to_datetime(season['end'])

            condition = (
                (df['building_number'] == building_id) &
                (df['date_time'] >= start_date) &
                (df['date_time'] <= end_date)
            )
            df.loc[condition, 'is_holiday_research'] = 1

In [ ]:
# ==========================================================
# 올림픽 피처
# ==========================================================

for df in [train, test]:
    # 전체 올림픽 기간에 대한 플래그
    df['is_olympic_period'] = 0
    start_date = pd.to_datetime('2024-07-26')
    end_date = pd.to_datetime('2024-08-11 23:59:59')

    olympic_condition = (df['date_time'] >= start_date) & (df['date_time'] <= end_date)
    df.loc[olympic_condition, 'is_olympic_period'] = 1

    # 올림픽 기간 내에서 특정 시간대 피처
    df['is_olympic_night'] = 0
    night_condition = (df['is_olympic_period'] == 1) & (df['hour'].isin([2, 3, 4, 5, 6]))
    df.loc[night_condition, 'is_olympic_night'] = 1

    df['is_olympic_day'] = 0
    day_condition = (df['is_olympic_period'] == 1) & (df['hour'].isin([16, 17, 18, 19]))
    df.loc[day_condition, 'is_olympic_day'] = 1

    # 주말 야간 피처 생성 (day_of_week: 금=4, 토=5)
    df['is_olympic_weekend_night'] = 0
    weekend_night_condition = (df['is_olympic_night'] == 1) & (df['day_of_week'].isin([4, 5]))
    df.loc[weekend_night_condition, 'is_olympic_weekend_night'] = 1

print("생성된 올림픽 관련 피처:")
print(test[test['is_olympic_period'] == 1][['date_time', 'is_olympic_day', 'is_olympic_night', 'is_olympic_weekend_night']].head())

In [ ]:
# ==========================================================
# CDH, THI, WCT 피처
# ==========================================================

def CDH(xs):
    cumsum = np.cumsum(xs - 26)
    return np.concatenate((cumsum[:11], cumsum[11:] - cumsum[:-11]))

def calculate_and_add_cdh(dataframe):
    cdhs = []
    for i in range(1, 101):
        temp = dataframe[dataframe['building_number'] == i]['temperature'].values
        cdh = CDH(temp)
        cdhs.append(cdh)
    return np.concatenate(cdhs)

train['CDH'] = calculate_and_add_cdh(train)
test['CDH'] = calculate_and_add_cdh(test)

train['THI'] = 9/5*train['temperature'] - 0.55*(1-train['humidity']/100)*(9/5*train['humidity']-26)+32
test['THI'] = 9/5*test['temperature'] - 0.55*(1-test['humidity']/100)*(9/5*test['humidity']-26)+32

train['WCT'] = 13.12 + 0.6125*train['temperature'] - 11.37*(train['windspeed']**0.16) + 0.3965*(train['windspeed']**0.16)*train['temperature']
test['WCT'] = 13.12 + 0.6125*test['temperature'] - 11.37*(test['windspeed']**0.16) + 0.3965*(test['windspeed']**0.16)*test['temperature']

In [ ]:
# ==========================================================
# 통계 피처 계산을 위한 이상치 제거
# ==========================================================
# ==========================================================
# 특정 기간 이상치 마킹
# ==========================================================

def mark_outlier_periods_for_stats(df):
    """
    reference/xgb_type.py에서 확인한 특정 기간들을 이상치로 마킹
    기존에는 데이터를 drop했지만, 여기서는 outlier_detect=1로 마킹

    기간 형식:
    - 날짜만: ['2024-07-07', '2024-07-08']  # 전체 날짜
    - 시간까지: ['2024-07-07 14:00', '2024-07-08 09:00']  # 특정 시간대
    """

    print("=== 특정 기간 이상치 마킹 시작 ===")

    # date_time 컬럼이 datetime 타입인지 확인하고 변환
    if not pd.api.types.is_datetime64_any_dtype(df['date_time']):
        print("date_time을 datetime 타입으로 변환 중...")
        df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d %H')

    initial_outliers = (df['outlier_detect'] == 1).sum()

    # 1. 건물기타 (Other Buildings) 등.
    other_buildings_config = [

    ]

    # 2. IDC(전화국) 등
    idc_config = [

    ]

    # 3. 병원 (Hospital) 등.
    hospital_config = [

    ]

    # 4. 상용 (Commercial) 등
    commercial_config = [

    ]

    # 5. 아파트 (Apartment) 등
    apartment_config = [

    ]

    # 6. 연구소 (Research Institute) 등.
    research_config = [
        {'building_id': 23, 'periods': [
            ['2024-06-07', '2024-06-07'],
            ['2024-08-16', '2024-08-16'],
        ]},
        {'building_id': 94, 'periods': [
            ['2024-06-07', '2024-06-07'],
            ['2024-08-16', '2024-08-16'],
        ]},

    ]

    # 7. 학교 (University) 등.
    university_config = [

    ]

    # 8. 호텔 (Hotel) 등
    hotel_config = [

    ]

    # 9. 공공 (Public) 등
    public_config = [
    ]

    # 10. 백화점 (Department Store) 등 (모든 휴무일 제외)
    department_store_config = [
        {'building_id': 19, 'periods': [
            ['2024-06-10', '2024-06-10'],
            ['2024-07-08', '2024-07-08'],
            ['2024-08-19', '2024-08-19'],
        ]},
        {'building_id': 27, 'periods': [
            ['2024-06-09', '2024-06-09'],
            ['2024-06-23', '2024-06-23'],
            ['2024-07-14', '2024-07-14'],
            ['2024-07-28', '2024-07-28'],
            ['2024-08-11', '2024-08-11'],
        ]},
        {'building_id': 29, 'periods': [
            ['2024-06-10', '2024-06-10'],
            ['2024-06-23', '2024-06-23'],
            ['2024-07-10', '2024-07-10'],
            ['2024-07-28', '2024-07-28'],
            ['2024-08-10', '2024-08-10'],
        ]},
        {'building_id': 32, 'periods': [
            ['2024-06-10', '2024-06-10'],
            ['2024-06-24', '2024-06-24'],
            ['2024-07-08', '2024-07-08'],
            ['2024-07-22', '2024-07-22'],
            ['2024-08-12', '2024-08-12'],
        ]},
        {'building_id': 40, 'periods': [
            ['2024-06-09', '2024-06-09'],
            ['2024-06-23', '2024-06-23'],
            ['2024-07-14', '2024-07-14'],
            ['2024-07-28', '2024-07-28'],
            ['2024-08-11', '2024-08-11'],
        ]},
        {'building_id': 45, 'periods': [
            ['2024-06-10', '2024-06-10'],
            ['2024-07-08', '2024-07-08'],
            ['2024-08-19', '2024-08-19'],
        ]},
        {'building_id': 54, 'periods': [
            ['2024-06-17', '2024-06-17'],
            ['2024-07-01', '2024-07-01'],
            ['2024-08-19', '2024-08-19'],
        ]},
        {'building_id': 59, 'periods': [
            ['2024-06-09', '2024-06-09'],
            ['2024-06-23', '2024-06-23'],
            ['2024-07-14', '2024-07-14'],
            ['2024-07-28', '2024-07-28'],
            ['2024-08-11', '2024-08-11'],
        ]},
        {'building_id': 63, 'periods': [
            ['2024-06-09', '2024-06-09'],
            ['2024-06-23', '2024-06-23'],
            ['2024-07-14', '2024-07-14'],
            ['2024-07-28', '2024-07-28'],
            ['2024-08-11', '2024-08-11'],
        ]},
        {'building_id': 74, 'periods': [
            ['2024-06-17', '2024-06-17'],
            ['2024-07-01', '2024-07-01'],
        ]},
        {'building_id': 79, 'periods': [
            ['2024-06-17', '2024-06-17'],
            ['2024-07-01', '2024-07-01'],
            ['2024-08-19', '2024-08-19'],
        ]},
        {'building_id': 95, 'periods': [
            ['2024-07-08', '2024-07-08'],
            ['2024-08-05', '2024-08-05'],
        ]},
    ]

    # 모든 설정을 통합 (10개 건물 타입 전체)
    all_configs = [
        ('건물기타 (Other Buildings)', other_buildings_config),
        ('IDC(전화국)', idc_config),
        ('병원 (Hospital)', hospital_config),
        ('상용 (Commercial)', commercial_config),
        ('아파트 (Apartment)', apartment_config),
        ('연구소 (Research Institute)', research_config),
        ('학교 (University)', university_config),
        ('호텔 (Hotel)', hotel_config),
        ('공공 (Public)', public_config),
        ('백화점 (Department Store)', department_store_config)
    ]

    total_marked = 0

    for building_type_name, configs in all_configs:
        print(f"\\n--- {building_type_name} 처리 ---")
        type_marked = 0

        if not configs:  # 빈 리스트인 경우
            print(f"  {building_type_name}: 설정된 이상치 기간 없음")
            continue

        for config in configs:
            building_id = config['building_id']
            periods = config['periods']

            building_marked = 0
            for period in periods:
                # 시간이 포함되어 있는지 확인
                if len(period[0].split()) > 1:  # 시간이 포함된 경우
                    start_datetime = pd.to_datetime(period[0])
                    end_datetime = pd.to_datetime(period[1])

                    # 정확한 시간까지 비교
                    condition = (df['building_number'] == building_id) & \
                               (df['date_time'] >= start_datetime) & \
                               (df['date_time'] <= end_datetime)

                    print(f"  건물 {building_id}: {period[0]} ~ {period[1]} (시간 포함)", end=" | ")
                else:  # 날짜만 있는 경우 (기존 방식)
                    start_date = pd.to_datetime(period[0]).date()
                    end_date = pd.to_datetime(period[1]).date()

                    # 해당 건물과 기간에 맞는 조건
                    condition = (df['building_number'] == building_id) & \
                               (df['date_time'].dt.date >= start_date) & \
                               (df['date_time'].dt.date <= end_date)

                    print(f"  건물 {building_id}: {period[0]} ~ {period[1]} (전체 날짜)", end=" | ")

                marked_count = condition.sum()
                df.loc[condition, 'outlier_detect'] = 1
                building_marked += marked_count
                print(f"{marked_count}개 마킹")

            type_marked += building_marked

        total_marked += type_marked
        print(f"  {building_type_name} 총 마킹: {type_marked}개")

    final_outliers = (df['outlier_detect'] == 1).sum()
    newly_marked = final_outliers - initial_outliers

    print(f"\\n=== 특정 기간 이상치 마킹 완료 ===")
    print(f"기존 이상치: {initial_outliers}개")
    print(f"새로 마킹된 이상치: {newly_marked}개")
    print(f"최종 이상치: {final_outliers}개")
    print(f"전체 데이터 대비 이상치 비율: {final_outliers/len(df)*100:.2f}%")

    return df

# date_time 컬럼을 먼저 datetime 타입으로 변환
print("=== 날짜 데이터 타입 확인 및 변환 ===")
if not pd.api.types.is_datetime64_any_dtype(train['date_time']):
    print("train 데이터의 date_time을 datetime 타입으로 변환 중...")
    train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')

if not pd.api.types.is_datetime64_any_dtype(test['date_time']):
    print("test 데이터의 date_time을 datetime 타입으로 변환 중...")
    test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

train_for_stats = train.copy()

# 전체 데이터에서 공휴일 제외
holiday_condition = (train_for_stats['date_time'].dt.month == 6) & (train_for_stats['date_time'].dt.day == 6) | \
                          (train_for_stats['date_time'].dt.month == 8) & (train_for_stats['date_time'].dt.day == 15)

train_for_stats.loc[holiday_condition, 'outlier_detect'] = 1

# train 데이터에 특정 기간 이상치 마킹 적용
train_for_stats = mark_outlier_periods_for_stats(train_for_stats)


train_for_stats = train_for_stats[train_for_stats['outlier_detect'] == 0].copy()

In [ ]:
# ==========================================================
# 통계 피처
# ==========================================================

# Calculate 'day_hour_mean'
power_mean = pd.pivot_table(train_for_stats, values='power_consumption', index=['building_number', 'hour', 'day_of_week'], aggfunc=np.mean).reset_index()
power_mean.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_mean']

power_month_mean = pd.pivot_table(train_for_stats, values='power_consumption', index=['building_number', 'month','hour', 'day_of_week'], aggfunc=np.mean).reset_index()
power_month_mean.columns = ['building_number','month', 'hour', 'day_of_week', 'monthly_day_hour_mean']

# Calculate 'day_hour_std'
power_std = pd.pivot_table(train_for_stats, values='power_consumption', index=['building_number', 'hour', 'day_of_week'], aggfunc=np.std).reset_index()
power_std.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_std']

# Calculate 'hour_mean'
power_hour_mean = pd.pivot_table(train_for_stats, values='power_consumption', index=['building_number', 'hour'], aggfunc=np.mean).reset_index()
power_hour_mean.columns = ['building_number', 'hour', 'hour_mean']

# Calculate 'hour_std'
power_hour_std = pd.pivot_table(train_for_stats, values='power_consumption', index=['building_number', 'hour'], aggfunc=np.std).reset_index()
power_hour_std.columns = ['building_number', 'hour', 'hour_std']

# Merge calculated features to 'train' and 'test' dataframes
train = train.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')

train = train.merge(power_month_mean, on=['building_number','month', 'hour', 'day_of_week'], how='left')
test = test.merge(power_month_mean, on=['building_number', 'month', 'hour', 'day_of_week'], how='left')

train = train.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')

train = train.merge(power_hour_mean, on=['building_number', 'hour'], how='left')
test = test.merge(power_hour_mean, on=['building_number', 'hour'], how='left')

train = train.merge(power_hour_std, on=['building_number', 'hour'], how='left')
test = test.merge(power_hour_std, on=['building_number', 'hour'], how='left')

train = train.reset_index(drop=True)

In [ ]:
# ==========================================================
# 추가 피처
# ==========================================================

# 냉방면적/연면적 비율 피쳐 생성
train['cooling_area_ratio'] = train['cooling_area'] / train['total_area']
test['cooling_area_ratio'] = test['cooling_area'] / test['total_area']

train['cooling_area_ratio'] = train['cooling_area_ratio'].fillna(0)
test['cooling_area_ratio'] = test['cooling_area_ratio'].fillna(0)

train['cooling_area_ratio'] = train['cooling_area_ratio'].replace([np.inf, -np.inf], 0)
test['cooling_area_ratio'] = test['cooling_area_ratio'].replace([np.inf, -np.inf], 0)

print(train['cooling_area_ratio'].head())

# PV용량 X 기온 (피쳐)
train['pv_temp_interaction'] = train['solar_power_capacity'] * train['temperature']
test['pv_temp_interaction'] = test['solar_power_capacity'] * test['temperature']

print(train['pv_temp_interaction'].head())

In [ ]:
# ==========================================================
# 모델 학습을 위한 데이터 준비 (outlier_detect==1 제거)
# ==========================================================

print("=== 모델 학습용 데이터 준비 ===")
print(f"원본 train 데이터 개수: {len(train)}")
print(f"outlier_detect==1인 데이터 개수: {(train['outlier_detect'] == 1).sum()}")

# outlier_detect==1인 행들을 제거하고 학습용 데이터 생성
train_clean = train[train['outlier_detect'] == 0].copy()

print(f"학습용 train 데이터 개수: {len(train_clean)}")
print(f"제거된 이상치 비율: {((len(train) - len(train_clean)) / len(train) * 100):.2f}%")

# 피처와 타겟 분리
X = train_clean.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity',
                     'power_consumption', 'sunshine', 'solar_radiation',
                     'hour', 'day_of_week','day','month','date_time', 'outlier_detect'],axis=1)

Y = train_clean[['building_type','power_consumption']]

test_X = test.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity',
                   'hour', 'day_of_week','month','day','date_time', 'outlier_detect'], axis=1)

print(f"피처 개수: {len(X.columns)}")
print(f"건물 타입: {sorted(X['building_type'].unique())}")

In [ ]:
# ==========================================================
# XGBoost 모델 학습 (Building Type별)
# ==========================================================

from sklearn.metrics import r2_score

RANDOM_SEED = 42
KFOLD_SPLITS = 12

type_list = X["building_type"].unique()

answer_df = pd.DataFrame(index=test_X.index, columns=["answer"], dtype=float)
pred_df   = pd.DataFrame(index=X.index,       columns=["pred"],   dtype=float)

# 각 building_type별로 최적 파라미터를 저장할 딕셔너리
best_params_by_type = {}

results_by_type = []

for btype in type_list:
    x  = X[X['building_type'] == btype].copy()

    y_orig = Y[Y['building_type'] == btype]['power_consumption'].copy()
    y = np.log1p(y_orig)

    xt = test_X[test_X['building_type'] == btype].copy()

    x  = pd.get_dummies(x,  columns=["building_number"], drop_first=False)
    xt = pd.get_dummies(xt, columns=["building_number"], drop_first=False)
    xt = xt.reindex(columns=x.columns, fill_value=0)

    x  = x.drop(columns=["building_type"])
    xt = xt.drop(columns=["building_type"])

    # K-fold로 최적 파라미터 찾기
    best_score = float('inf')
    best_params = None

    # 여러 파라미터 조합 시도
    param_combinations = [
        {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 1},
        {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 2},
        {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 3},
        # {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 5},
        # {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 10},
        # {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 50},
        # {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.03, 'subsample': 0.65, 'colsample_bytree': 0.45, 'min_child_weight': 4, 'alpha': 100},
    ]

    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_SEED)

    for params in param_combinations:
        fold_scores = []

        for tr_idx, va_idx in kf.split(x.values):
            X_tr, X_va = x.values[tr_idx], x.values[va_idx]
            y_tr, y_va = y.values[tr_idx], y.values[va_idx]
            y_tr_orig, y_va_orig = y_orig.values[tr_idx], y_orig.values[va_idx]

            model = XGBRegressor(
                learning_rate       = params['learning_rate'],
                n_estimators       = params['n_estimators'],
                max_depth          = params['max_depth'],
                subsample          = params['subsample'],
                colsample_bytree   = params['colsample_bytree'],
                min_child_weight   = params['min_child_weight'],
                random_state       = RANDOM_SEED,
                objective          = weighted_mse(params.get('alpha', 3)),
                early_stopping_rounds = 100,
                eval_metric        = custom_smape,
            )

            model.fit(
                X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                verbose=0,  # K-fold 중에는 verbose 끄기
            )

            y_va_pred_log = model.predict(X_va)
            y_va_pred = np.maximum(0, np.expm1(y_va_pred_log))

            va_smape = smape(y_va_orig, y_va_pred)
            fold_scores.append(va_smape)

        avg_score = np.mean(fold_scores)
        if avg_score < best_score:
            best_score = avg_score
            best_params = params

    # 최적 파라미터 저장
    best_params_by_type[btype] = best_params

    print(f"Building type = {btype} | Best params: {best_params} | Best CV SMAPE: {best_score:.4f}")

    # 최적 파라미터로 전체 데이터로 모델 훈련
    final_model = XGBRegressor(
        learning_rate       = best_params['learning_rate'],
        n_estimators       = best_params['n_estimators'],
        max_depth          = best_params['max_depth'],
        subsample          = best_params['subsample'],
        colsample_bytree   = best_params['colsample_bytree'],
        min_child_weight   = best_params['min_child_weight'],
        random_state       = RANDOM_SEED,
        objective          = weighted_mse(best_params['alpha']),
        eval_metric        = custom_smape,
    )

    # 전체 데이터로 훈련
    final_model.fit(x.values, y.values, verbose=500)

    # 훈련 데이터에 대한 예측 (검증용)
    train_pred_log = final_model.predict(x.values)
    train_pred = np.maximum(0, np.expm1(train_pred_log))
    pred_df.loc[x.index, "pred"] = train_pred

    # 테스트 데이터에 대한 예측
    test_pred_log = final_model.predict(xt.values)
    test_pred = np.maximum(0, np.expm1(test_pred_log))
    answer_df.loc[xt.index, "answer"] = test_pred

    # 최종 모델 성능 평가
    final_train_smape = smape(y_orig.values, train_pred)
    final_train_r2 = r2_score(y_orig.values, train_pred)

    results_by_type.append({
        "building_type": btype,
        "Best CV SMAPE": best_score,
        "Final Train SMAPE": final_train_smape,
        "Final Train R²": final_train_r2,
        "Best Params": best_params
    })

    print(f"Building type = {btype} | "
          f"Best CV SMAPE = {best_score:.4f} | "
          f"Final Train SMAPE = {final_train_smape:.4f} | "
          f"Final Train R² = {final_train_r2:.4f}")

    # Feature importance 시각화
    sorted_idx = final_model.feature_importances_.argsort()
    plt.figure(figsize=(8, 15))
    plt.barh(x.columns[sorted_idx], final_model.feature_importances_[sorted_idx])
    plt.xlabel(f"{btype} XGB Feature Importance")
    plt.title(f"{btype} (Final Model)")
    plt.tight_layout()
    plt.show()

# 전체 SMAPE (최종 모델 예측 기준)
total_smape = smape(
    Y.sort_index()["power_consumption"].values,
    pred_df.sort_index()["pred"].values
)
print(f"Total SMAPE = {total_smape:.4f}")

# 결과 요약
results_summary = pd.DataFrame(results_by_type)
print("\n=== Results Summary ===")
print(results_summary)

In [ ]:
# ==========================================================
# 결과 저장 및 시각화
# ==========================================================

# 결과 데이터프레임 상세 출력
results_df = pd.DataFrame(results_by_type)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
print("=== 상세 결과 ===")
print(results_df)

# 제출 파일 생성
submit = pd.read_csv("./data/raw/sample_submission.csv")
submit["answer"] = answer_df.loc[submit.index, "answer"]
submit['answer'] = submit['answer'].round(2)

# 현재 시간을 포함한 파일명으로 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"./data/output/xgb_lsh_{timestamp}.csv"
submit.to_csv(output_filename, index=False)
print(f"제출 파일 저장: {output_filename}")

# 예측 vs 실제 산점도
y_sorted = Y.sort_index()["power_consumption"]
pred_sorted = pred_df.sort_index()["pred"]

# 결측 제거 (예측 안 된 경우)
mask = ~pred_sorted.isna()
y_sorted = y_sorted[mask]
pred_sorted = pred_sorted[mask]

# 산점도
plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_sorted, y=pred_sorted, alpha=0.4)
plt.plot([0, max(y_sorted.max(), pred_sorted.max())],
         [0, max(y_sorted.max(), pred_sorted.max())],
         linestyle='--', color='red')
plt.xlabel("Actual Power Consumption")
plt.ylabel("Predicted")
plt.title(f"All Types - Predicted vs Actual (Total SMAPE: {total_smape:.4f})")
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"\n=== 최종 결과 요약 ===")
print(f"학습에 사용된 데이터: {len(train_clean):,}개 (이상치 {(len(train) - len(train_clean)):,}개 제거)")
print(f"전체 SMAPE: {total_smape:.4f}")
print(f"제출 파일: {output_filename}")

In [ ]:
# ==========================================================
# 예측 데이터 통합 및 저장
# ==========================================================

train_df = train
prediction_df = pd.read_csv(output_filename)

prediction_df = prediction_df.rename(columns={'answer': 'power_consumption'})


def parse_datetime_info(df):
    df['building_number'] = df['num_date_time'].str.split('_').str[0].astype(int)
    df['date_time'] = pd.to_datetime(
        df['num_date_time'].str.split('_').str[1].str[:8] +
        df['num_date_time'].str.split('_').str[1].str[9:11],
        format='%Y%m%d%H'
    )
    return df

prediction_df = parse_datetime_info(prediction_df.copy())

# 예측 데이터에 기존 데이터 구조 맞추기 (빈 값으로 채움)
for col in train_df.columns:
    if col not in prediction_df.columns:
        prediction_df[col] = None

prediction_df = prediction_df[train_df.columns]

merged_df = pd.concat([train_df, prediction_df], ignore_index=True)
merged_df.to_csv('data/processed/merged_data_prediction.csv', index=False)
print(f"데이터 연결 완료: {train_df.shape[0]} + {prediction_df.shape[0]} = {merged_df.shape[0]}행")